In [1]:
!nvidia-smi

Sat Dec 19 04:54:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get Data

In [3]:

import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

alzheimers-dataset-4-class-of-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip -qq /content/alzheimers-dataset-4-class-of-images.zip


# Dependencies

In [10]:
!pip install -qq efficientnet_pytorch torchtoolbox
!pip install -qq pytorch_ranger


In [11]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import gc
import os
import time
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from efficientnet_pytorch import EfficientNet
import cv2
from PIL import Image, ImageFilter
from albumentations.pytorch import ToTensor
import albumentations as albu
from pytorch_ranger import Ranger
%matplotlib inline



In [13]:
# At least fixing some random seeds. 
# It is still impossible to make results 100% reproducible when using GPU
warnings.simplefilter('ignore')
torch.manual_seed(47)
np.random.seed(47)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
!mkdir checkpoint
train_df = pd.read_csv('train.csv')

mkdir: cannot create directory ‘checkpoint’: File exists


# Dataloader


In [14]:
class Dataset(Dataset):
    def __init__(self, df: pd.DataFrame, train: bool = True, transforms = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            data (np.ndarray): resized images data in a shape of (HxWxC)
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            
        """
        self.df = df
        
        self.transforms = transforms
        self.train = train
        
    def __getitem__(self, index):
        name = self.df.iloc[index,1]
        
        if self.train:
          if name[0:4]=='mild':
            path = 'Alzheimer_s Dataset/train/MildDemented/' + str(name)
          elif name[0:4]=='very':
            path='Alzheimer_s Dataset/train/VeryMildDemented/' + str(name)
          elif name[0:4]=='nonD':
            path='Alzheimer_s Dataset/train/NonDemented/' + str(name)   
          else:
            path= 'Alzheimer_s Dataset/train/ModerateDemented/' + str(name)  
        else:
          path = 'test/' + str(name)
          
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)

        if self.transforms:
            image = self.transforms(image=np.array(image))
            image=image['image']
            
        if self.train:
            y = self.df.loc[index]['label']
            return image, y
        else:
            return image
    
    def __len__(self):
        return len(self.df)
    
    


Transforms

In [15]:
train_transform = albu.Compose([
               albu.HorizontalFlip(),
               albu.VerticalFlip(),
               albu.Cutout(4),
               albu.Rotate(limit=30),
               albu.Resize(256,256),
               albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                ToTensor()
            ])
test_transform = albu.Compose([
                albu.Resize(256,256),
                albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                ToTensor()
            ])

In [16]:
  # Going to use efficientnet-b0 NN architecture
skf = StratifiedKFold(n_splits=5, random_state=47, shuffle=True)

In [17]:
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffling the train data
train_df

,Unnamed: 0,name,label
0,1590,verymildDem444.jpg,1
1,3919,nonDem1218.jpg,0
2,4510,nonDem2019.jpg,0
3,4688,nonDem97.jpg,0
4,4897,nonDem2287.jpg,0
...,...,...,...
5116,3095,nonDem1125.jpg,0
5117,2896,nonDem2278.jpg,0
5118,691,mildDem582.jpg,1
5119,3336,nonDem1494.jpg,0


# Training

In [18]:
# Number of epochs to run
model_path = 'model.pth'  # Path and filename to save model to
es_patience = 5  # Early Stopping patience - for how many epochs with no improvements to wait

oof = np.zeros((len(train_df), 1))  # Out Of Fold predictions

# We stratify by target value, thus, according to sklearn StratifiedKFold documentation
# We can fill `X` with zeroes of corresponding length to use it as a placeholder
# since we only need `y` to stratify the data
def train_model(model,criterion,optimizer,scheduler,epochs=25):
    for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(len(train_df)), y=train_df['label']), 1):
    
        if fold==1:
            print('=' * 20, 'Fold', fold, '=' * 20)
            best_val = None  # Best validation score within this fold
            patience = es_patience  # Current patience counter
        
    
            train = Dataset(df=train_df.iloc[train_idx].reset_index(drop=True), train=True, transforms=train_transform)
            val = Dataset(df=train_df.iloc[val_idx].reset_index(drop=True),  train=True, transforms=test_transform)  
    
            train_loader = DataLoader(dataset=train, batch_size=128, shuffle=True)   
            val_loader = DataLoader(dataset=val, batch_size=64, shuffle=False)
       #     test_loader = DataLoader(dataset=test, batch_size=64, shuffle=False)
            val_roc=0
            for epoch in range(epochs):
                start_time = time.time()
                correct = 0
                epoch_loss = 0
                scheduler.step(val_roc)
                model.train()
        
                for x, y in train_loader:
                    x = torch.tensor(x, device=device, dtype=torch.float32)
                    y = torch.tensor(y, device=device, dtype=torch.float32)
            
                    optimizer.zero_grad()
            
                    z = model(x)
                    loss = criterion(z, y.unsqueeze(1))
                    loss.backward()
                    optimizer.step()
                    pred = torch.round(torch.sigmoid(z))  # round off sigmoid to obtain predictions
                    correct += (pred.cpu() == y.cpu().unsqueeze(1)).sum().item()  # tracking number of correctly predicted samples
                    epoch_loss += loss.item()
                train_acc = correct / len(train_idx)

                model.eval()  # switch model to the evaluation mode
                val_preds = torch.zeros((len(val_idx), 1), dtype=torch.float32, device=device)
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for j, (x_val, y_val) in enumerate(val_loader):
                        x_val = torch.tensor(x_val, device=device, dtype=torch.float32)
                        y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
                        z_val = model(x_val)
                        val_pred = torch.sigmoid(z_val)
                        val_preds[j*x_val.shape[0]:j*x_val.shape[0] + x_val.shape[0]] = val_pred
                    val_acc = accuracy_score(train_df.iloc[val_idx]['label'].values, torch.round(val_preds.cpu()))
                    val_roc = roc_auc_score(train_df.iloc[val_idx]['label'].values, val_preds.cpu())
            
                    print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | Training time: {}'.format(
                    epoch + 1, 
                    epoch_loss, 
                    train_acc, 
                    val_acc, 
                    val_roc, 
                    str(datetime.timedelta(seconds=time.time() - start_time))))
            
                    # During the first iteration (first epoch) best validation is set to None
                    if not best_val:
                        best_val = val_roc  # So any validation roc_auc we have is the best one for now
                        torch.save(model, model_path)  # Saving the model
                        continue
                
                    if val_roc >= best_val:
                        best_val = val_roc
                        patience = es_patience  # Resetting patience since we have new best validation accuracy
                        torch.save(model, model_path)  # Saving current best model
                        torch.save(model.state_dict(),'./checkpoint/current_checkpoint.pt')
                    else:
                        patience -= 1
                        if patience == 0:
                            print('Early stopping. Best Val roc_auc: {:.3f}'.format(best_val))
                            break
        else :
            break
   

Freeze all the layers except the last for few epochs. helps in fatser convergence of the model

In [23]:
model = EfficientNet.from_pretrained('efficientnet-b0')
for param in model.parameters():
    param.requires_grad = False

in_features = model._fc.in_features
classifier =nn.Linear(in_features, 1)
model._fc=classifier

if torch.cuda.is_available():
    model = model.cuda()
     
optimizer = Ranger(model.parameters(), lr=0.001,weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=4, verbose=False, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
weight = torch.tensor([32542])/1168
weight=weight.cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [24]:
model=train_model(model,criterion,optimizer,scheduler,epochs=4)

==================== Fold 1 ====================
Epoch 001: | Loss: 255.158 | Train acc: 0.506 | Val acc: 0.501 | Val roc_auc: 0.566 | Training time: 0:00:19.937381
Epoch 002: | Loss: 214.429 | Train acc: 0.503 | Val acc: 0.501 | Val roc_auc: 0.595 | Training time: 0:00:19.599693
Epoch 003: | Loss: 171.860 | Train acc: 0.501 | Val acc: 0.501 | Val roc_auc: 0.610 | Training time: 0:00:19.612824
Epoch 004: | Loss: 140.449 | Train acc: 0.500 | Val acc: 0.501 | Val roc_auc: 0.631 | Training time: 0:00:19.600694


Unfreeze all the layers

In [25]:
model = EfficientNet.from_pretrained('efficientnet-b0')
for param in model.parameters():
    param.requires_grad = True

in_features = model._fc.in_features
classifier =nn.Linear(in_features, 1)
model._fc=classifier

if torch.cuda.is_available():
    model = model.cuda()
model.load_state_dict(torch.load('./checkpoint/current_checkpoint.pt'),strict=True)    
optimizer = Ranger(model.parameters(), lr=0.001,weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=4, verbose=False, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
weight = torch.tensor([32542])/1168
weight=weight.cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)


Loaded pretrained weights for efficientnet-b0


In [26]:
model=train_model(model,criterion,optimizer,scheduler,epochs=10)

==================== Fold 1 ====================
Epoch 001: | Loss: 90.998 | Train acc: 0.499 | Val acc: 0.501 | Val roc_auc: 0.706 | Training time: 0:00:35.246843
Epoch 002: | Loss: 57.069 | Train acc: 0.517 | Val acc: 0.501 | Val roc_auc: 0.756 | Training time: 0:00:35.078471
Epoch 003: | Loss: 49.086 | Train acc: 0.568 | Val acc: 0.622 | Val roc_auc: 0.782 | Training time: 0:00:35.196994
Epoch 004: | Loss: 41.384 | Train acc: 0.643 | Val acc: 0.743 | Val roc_auc: 0.825 | Training time: 0:00:35.204285
Epoch 005: | Loss: 31.559 | Train acc: 0.710 | Val acc: 0.737 | Val roc_auc: 0.847 | Training time: 0:00:35.121698
Epoch 006: | Loss: 21.004 | Train acc: 0.804 | Val acc: 0.777 | Val roc_auc: 0.872 | Training time: 0:00:35.047263
Epoch 007: | Loss: 14.731 | Train acc: 0.883 | Val acc: 0.765 | Val roc_auc: 0.890 | Training time: 0:00:35.124234
Epoch 008: | Loss: 10.402 | Train acc: 0.918 | Val acc: 0.764 | Val roc_auc: 0.911 | Training time: 0:00:35.612751
Epoch 009: | Loss: 9.396 | Trai